# Credit 

@yekenot Vladimir Demidov https://www.kaggle.com/code/yekenot/llm-detect-by-regression/notebook

 @ilhansevval - https://www.kaggle.com/code/ilhansevval/ensemble-learning-with-voting-classifier-3
    
https://www.kaggle.com/code/xiaocao123/ai-generated-text-detection-add-new-data

https://www.kaggle.com/code/rsuhara/ai-generated-text-detection-quick-baseline

Inspired by VLADIMIR DEMIDOV's work :
https://www.kaggle.com/code/yekenot/llm-detect-by-regression
https://www.kaggle.com/code/x75a40890/ai-generated-text-detection-quick-baseline

Using new train dataset https://www.kaggle.com/datasets/thedrcat/daigt-v2-train-dataset/

@chenbaoying https://www.kaggle.com/code/chenbaoying/0-911-ai-generated-text-detection-test-feature#Voting-Classifier

 

In [1]:
import regex as re
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import random 

In [2]:
random.seed(42)
np.random.seed(42)

## Using dataset with artificially introduced typos to mimick the hidden test data

In [3]:
train = pd.read_csv("/kaggle/input/ai-essay-detection-daigt-v2-dataset-with-typos/daigt-v2-train-dataset-with-typos.csv")
# train = pd.read_csv('/kaggle/input/ai-essay-detection-daigt-v2-dataset-with-typos/daigt-v2-train-dataset-without-typos.csv')
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train['text'] = train['text'].str.replace('\n', '')
test['text'] = test['text'].str.replace('\n', '') 
train.label.value_counts()

label
0    14250
1    14250
Name: count, dtype: int64

In [4]:
# train1 = train.groupby('label').apply(lambda x: x.sample(150000)).reset_index(drop=True)
# train1.label.value_counts()

In [5]:
## credit @nbroad, @phucpx

student_df = train[train['label'] == 0]
llm_df = train[train['label'] == 1]

all_human = set(list(''.join(student_df.text.to_list())))
other = set(list(''.join(llm_df.text.to_list())))
chars_to_remove = ''.join([x for x in other if x not in all_human])
print(chars_to_remove)

translation_table = str.maketrans('', '', chars_to_remove)
def remove_chars(s):
    return s.translate(translation_table) 
train['text'] = train['text'].apply(remove_chars)

@🏙🏽👥🤗😩护🙌😷中💨🔥🍕’🏕禁💅🙄驶🏠🌞—🏡🥦🧹о🎬⚽💔🌿🧡🍿🕒”😕🔬🏋🎶🤪🚔з🤔😊÷司🥪🏏🍟🛋💇🤣🐶机🎠上💡🏞♀🌏📞📝有🥨🔑🌱🎤🥕🕵🔋用🏟🏀🍔🍄🎢🏖🏳😔🦁必🌸🧠–‍📄🏼🍝🎸🐳💆。💰👫🤦👻🤓‘🧙👇🏨意🧩😡🐠🙈🍁🏆💭🎉🌯都😆💻🤢该🏛👍😒🦎🤷🤟💘时👂ü📣😳😱п🇸


In [6]:
%%time

df = pd.concat([train['text'], test['text']])

vectorizer = TfidfVectorizer(sublinear_tf=True,
                             ngram_range=(3, 4),
#                              tokenizer=lambda x: re.findall(r'[^\W]+', x),
#                              token_pattern=None,
                             token_pattern = r'(?u)\b\w+\b|\b\w\b',
                             strip_accents='unicode',
                             )

vectorizer = vectorizer.fit(test.text)
X = vectorizer.transform(df)

CPU times: user 28.4 s, sys: 3.99 ms, total: 28.4 s
Wall time: 28.4 s


In [7]:
%%time
lr_model = LogisticRegression()
sgd_model = SGDClassifier(max_iter=5000, loss="modified_huber", random_state=42)

ensemble = VotingClassifier(estimators=[('lr', lr_model),
                                        ('sgd', sgd_model),
                                       ],
                            weights=[0.01, 0.99],
                            voting='soft'
                           )
ensemble.fit(X[:train.shape[0]], train.label)

CPU times: user 25.9 ms, sys: 3.02 ms, total: 28.9 ms
Wall time: 34.1 ms


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=5000, random_state=42))],
                 voting='soft', weights=[0.01, 0.99])

In [8]:
preds_test = ensemble.predict_proba(X[train.shape[0]:])[:, 1]
pd.DataFrame({'id':test["id"], 'generated':preds_test}).to_csv('submission.csv', index=False)

In [9]:
pd.read_csv('/kaggle/working/submission.csv')

,id,generated
0,0000aaaa,0.506984
1,1111bbbb,0.506984
2,2222cccc,0.506984
